# Sprint 12. Pipelines, grid search i text mining

## Nivell 1

## Exercici 1.  Agafa el conjunt de dades que vulguis i realitza un pipeline i un gridsearch aplicant l'algoritme de Random Forest.



In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

In [3]:
df = pd.read_csv("../Sampling/Climbers.csv", sep=";")

#### Canviem el nom de les columnes per fer la manipulació més fàcilment

In [4]:
df.columns = ['Timestamp', 'Sex', 'Height', 'Weight', 'Arm_span','Time_climbing', 'Indoor_vs_outdoor',
              'Hardest_V_Grade_ever','Hardest_V_Grade_last3months','V_Grade_sent_90perc_routes',
              'Hardest_Ewbank_Grade_ever','Hardest_Ewbank_Grade_last3months','Ewbank_Grade_sent_90perc_routes',
              'Freq_climbing_sessions_w','Avg_hours_climbing_w','Avg_hours_training_for_climbing_w',
              'Freq_Hangboard_w', 'Hangboard_grips_used','Style_Hangboard','Max_Weight_Hangboard_18mm_Half_crimp',
              'Max_Weight_Hangboard_18mm_Open_crimp','Min_Edge_mm,kg_Half_crimp','Min_Edge_mm,kg_Open_crimp',
              'Freq_CampusBoard_w', 'Time_CampusBoard_h','Freq_Endurance_session_w','Endurance_training', 
              'Freq_General_Strength_Training_w','Time_General_Strength_Training_h','Type_StrengthTraining', 
              'Other_activities','Max_PullUp_reps', 'Max_weighted_PullUps_5rep', 'Max_PushUp_reps',
              'Max_L-sit_time']

#### Convertim les columnes objecte a nombre

In [6]:
df['Sex'] = df['Sex'].astype('category').cat.codes # 0 = Female, 1 = Male
df.Height = pd.to_numeric(df.Height, errors='coerce')
df.Weight = pd.to_numeric(df.Weight, errors='coerce')
df.Arm_span = pd.to_numeric(df.Arm_span, errors='coerce')
df.Hardest_Ewbank_Grade_ever = pd.to_numeric(df.Hardest_Ewbank_Grade_ever, errors='coerce')
df.Hardest_Ewbank_Grade_last3months = pd.to_numeric(df.Hardest_Ewbank_Grade_last3months, errors='coerce')
df.Ewbank_Grade_sent_90perc_routes = pd.to_numeric(df.Ewbank_Grade_sent_90perc_routes, errors='coerce')

In [7]:
def createTime(time):
    start = time.find("- ") + len("- ")
    end = time.find(" years")
    substring = time[start:end]
    return int(math.ceil(float(substring)))

In [8]:
df['Time_climbing'] = df['Time_climbing'].replace('More than 15 years', '- 15 years')
df['Time_climbing'] = df['Time_climbing'].apply(lambda x: createTime(x))

#### Seleccionem les columnes numèriques únicament

In [9]:
df_no_object = df.select_dtypes(exclude=['object'])
df_no_object = df_no_object.dropna()

In [10]:
df_no_object

,Sex,Height,Weight,Arm_span,Time_climbing,Hardest_Ewbank_Grade_ever,Hardest_Ewbank_Grade_last3months,Ewbank_Grade_sent_90perc_routes,Freq_climbing_sessions_w,Avg_hours_climbing_w,Avg_hours_training_for_climbing_w,Freq_Hangboard_w,Freq_CampusBoard_w,Time_CampusBoard_h,Freq_Endurance_session_w,Freq_General_Strength_Training_w,Time_General_Strength_Training_h
2,1,178.0,67.0,175.0,1,21.0,21.0,19.0,3,4,3,2,0,0,2,3,2
3,1,173.0,70.0,178.0,10,21.0,21.0,18.0,2,6,1,1,0,0,1,0,0
7,1,186.0,88.0,192.0,2,23.0,22.0,21.0,3,8,8,1,0,0,2,3,4
8,1,185.0,60.0,180.0,1,21.0,21.0,18.0,1,3,1,0,0,0,0,1,1
10,1,178.0,81.6,178.0,6,25.0,23.0,21.0,2,4,0,1,0,0,2,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
584,1,172.0,82.5,191.7,3,22.0,22.0,19.0,2,4,2,1,0,0,0,5,10
585,1,165.0,63.0,165.0,9,31.0,31.0,30.0,4,12,4,1,1,1,0,1,1
586,1,164.0,68.0,164.0,5,29.0,29.0,25.0,2,3,4,3,0,0,2,1,1
587,1,178.0,65.0,182.0,4,27.0,26.0,23.0,3,4,2,7,0,0,1,0,0


#### Eliminem els outliers

In [11]:
df_no_object.Height.describe()

count     393.000000
mean      178.840992
std        58.779576
min         1.680000
25%       172.000000
50%       178.000000
75%       183.000000
max      1295.000000
Name: Height, dtype: float64

In [12]:
df_no_object.Height.value_counts().sort_values()

159.0      1
62.0       1
1295.0     1
153.0      1
182.9      1
          ..
170.0     17
175.0     22
183.0     24
180.0     34
178.0     41
Name: Height, Length: 67, dtype: int64

In [13]:
df_no_object = df_no_object.drop(df_no_object[df_no_object['Height'] > 200].index)
df_no_object = df_no_object.drop(df_no_object[df_no_object['Height'] < 111].index)

In [14]:
df_no_object.Weight.describe()

count    388.000000
mean      72.415182
std       15.691350
min       42.000000
25%       65.000000
50%       70.300000
75%       77.025000
max      180.000000
Name: Weight, dtype: float64

In [15]:
df_no_object.Weight.value_counts().sort_values()

71.2      1
175.0     1
64.5      1
180.0     1
82.5      1
         ..
72.0     16
75.0     17
65.0     20
70.0     23
68.0     28
Name: Weight, Length: 97, dtype: int64

In [16]:
df_no_object = df_no_object.drop(df_no_object[df_no_object['Weight'] > 120].index)

In [17]:
df_no_object.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 380 entries, 2 to 590
Data columns (total 17 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Sex                                380 non-null    int8   
 1   Height                             380 non-null    float64
 2   Weight                             380 non-null    float64
 3   Arm_span                           380 non-null    float64
 4   Time_climbing                      380 non-null    int64  
 5   Hardest_Ewbank_Grade_ever          380 non-null    float64
 6   Hardest_Ewbank_Grade_last3months   380 non-null    float64
 7   Ewbank_Grade_sent_90perc_routes    380 non-null    float64
 8   Freq_climbing_sessions_w           380 non-null    int64  
 9   Avg_hours_climbing_w               380 non-null    int64  
 10  Avg_hours_training_for_climbing_w  380 non-null    int64  
 11  Freq_Hangboard_w                   380 non-null    int64  

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math

In [2]:
df = pd.read_csv("../Sampling/Climbers.csv", sep=";")

#### Canviem el nom de les columnes per fer la manipulació més fàcilment

In [3]:
df.columns = ['Timestamp', 'Sex', 'Height', 'Weight', 'Arm_span','Time_climbing', 'Indoor_vs_outdoor',
              'Hardest_V_Grade_ever','Hardest_V_Grade_last3months','V_Grade_sent_90perc_routes',
              'Hardest_Ewbank_Grade_ever','Hardest_Ewbank_Grade_last3months','Ewbank_Grade_sent_90perc_routes',
              'Freq_climbing_sessions_w','Avg_hours_climbing_w','Avg_hours_training_for_climbing_w',
              'Freq_Hangboard_w', 'Hangboard_grips_used','Style_Hangboard','Max_Weight_Hangboard_18mm_Half_crimp',
              'Max_Weight_Hangboard_18mm_Open_crimp','Min_Edge_mm,kg_Half_crimp','Min_Edge_mm,kg_Open_crimp',
              'Freq_CampusBoard_w', 'Time_CampusBoard_h','Freq_Endurance_session_w','Endurance_training', 
              'Freq_General_Strength_Training_w','Time_General_Strength_Training_h','Type_StrengthTraining', 
              'Other_activities','Max_PullUp_reps', 'Max_weighted_PullUps_5rep', 'Max_PushUp_reps',
              'Max_L-sit_time']

#### Convertim les columnes objecte a nombre

In [4]:
df['Sex'] = df['Sex'].astype('category').cat.codes # 0 = Female, 1 = Male
df.Height = pd.to_numeric(df.Height, errors='coerce')
df.Weight = pd.to_numeric(df.Weight, errors='coerce')
df.Arm_span = pd.to_numeric(df.Arm_span, errors='coerce')
df.Hardest_Ewbank_Grade_ever = pd.to_numeric(df.Hardest_Ewbank_Grade_ever, errors='coerce')
df.Hardest_Ewbank_Grade_last3months = pd.to_numeric(df.Hardest_Ewbank_Grade_last3months, errors='coerce')
df.Ewbank_Grade_sent_90perc_routes = pd.to_numeric(df.Ewbank_Grade_sent_90perc_routes, errors='coerce')

In [5]:
def createTime(time):
    start = time.find("- ") + len("- ")
    end = time.find(" years")
    substring = time[start:end]
    return int(math.ceil(float(substring)))

In [6]:
df['Time_climbing'] = df['Time_climbing'].replace('More than 15 years', '- 15 years')
df['Time_climbing'] = df['Time_climbing'].apply(lambda x: createTime(x))

#### Seleccionem les columnes numèriques únicament

In [7]:
df_no_object = df.select_dtypes(exclude=['object'])
df_no_object = df_no_object.dropna()

In [8]:
df_no_object

,Sex,Height,Weight,Arm_span,Time_climbing,Hardest_Ewbank_Grade_ever,Hardest_Ewbank_Grade_last3months,Ewbank_Grade_sent_90perc_routes,Freq_climbing_sessions_w,Avg_hours_climbing_w,Avg_hours_training_for_climbing_w,Freq_Hangboard_w,Freq_CampusBoard_w,Time_CampusBoard_h,Freq_Endurance_session_w,Freq_General_Strength_Training_w,Time_General_Strength_Training_h
2,1,178.0,67.0,175.0,1,21.0,21.0,19.0,3,4,3,2,0,0,2,3,2
3,1,173.0,70.0,178.0,10,21.0,21.0,18.0,2,6,1,1,0,0,1,0,0
7,1,186.0,88.0,192.0,2,23.0,22.0,21.0,3,8,8,1,0,0,2,3,4
8,1,185.0,60.0,180.0,1,21.0,21.0,18.0,1,3,1,0,0,0,0,1,1
10,1,178.0,81.6,178.0,6,25.0,23.0,21.0,2,4,0,1,0,0,2,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
584,1,172.0,82.5,191.7,3,22.0,22.0,19.0,2,4,2,1,0,0,0,5,10
585,1,165.0,63.0,165.0,9,31.0,31.0,30.0,4,12,4,1,1,1,0,1,1
586,1,164.0,68.0,164.0,5,29.0,29.0,25.0,2,3,4,3,0,0,2,1,1
587,1,178.0,65.0,182.0,4,27.0,26.0,23.0,3,4,2,7,0,0,1,0,0


#### Eliminem els outliers

In [9]:
df_no_object.Height.describe()

count     393.000000
mean      178.840992
std        58.779576
min         1.680000
25%       172.000000
50%       178.000000
75%       183.000000
max      1295.000000
Name: Height, dtype: float64

In [10]:
df_no_object.Height.value_counts().sort_values()

159.0      1
62.0       1
1295.0     1
153.0      1
182.9      1
          ..
170.0     17
175.0     22
183.0     24
180.0     34
178.0     41
Name: Height, Length: 67, dtype: int64

In [11]:
df_no_object = df_no_object.drop(df_no_object[df_no_object['Height'] > 200].index)
df_no_object = df_no_object.drop(df_no_object[df_no_object['Height'] < 111].index)

In [12]:
df_no_object.Weight.describe()

count    388.000000
mean      72.415182
std       15.691350
min       42.000000
25%       65.000000
50%       70.300000
75%       77.025000
max      180.000000
Name: Weight, dtype: float64

In [13]:
df_no_object.Weight.value_counts().sort_values()

71.2      1
175.0     1
64.5      1
180.0     1
82.5      1
         ..
72.0     16
75.0     17
65.0     20
70.0     23
68.0     28
Name: Weight, Length: 97, dtype: int64

In [14]:
df_no_object = df_no_object.drop(df_no_object[df_no_object['Weight'] > 120].index)

In [18]:
df_no_object.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 380 entries, 2 to 590
Data columns (total 17 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Sex                                380 non-null    int8   
 1   Height                             380 non-null    float64
 2   Weight                             380 non-null    float64
 3   Arm_span                           380 non-null    float64
 4   Time_climbing                      380 non-null    int64  
 5   Hardest_Ewbank_Grade_ever          380 non-null    float64
 6   Hardest_Ewbank_Grade_last3months   380 non-null    float64
 7   Ewbank_Grade_sent_90perc_routes    380 non-null    float64
 8   Freq_climbing_sessions_w           380 non-null    int64  
 9   Avg_hours_climbing_w               380 non-null    int64  
 10  Avg_hours_training_for_climbing_w  380 non-null    int64  
 11  Freq_Hangboard_w                   380 non-null    int64  

### Train test split
Agafem 'time climbing' com a target per perdir!


In [19]:
X = df_no_object.loc[:, df_no_object.columns != 'Time_climbing']
y = df_no_object.loc[:,['Time_climbing']]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.2)

### Pipeline amb gridsearch i Random Forest
Per poder incloure funcions en un pipeline és important que tinguin un mètode fit i un transform. Per tant, podem incloure el Standard Scaler, PCA i el Random Forest en el pipeline. 

In [28]:
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

# Define a pipeline to search for the best combination of PCA truncation
# and classifier regularization.
pca = PCA()
# Define a Standard Scaler to normalize inputs
scaler = StandardScaler()
# Define regressor
regressor = RandomForestRegressor()

#Create pipeline
pipe = Pipeline(steps=[("scaler", scaler), ("pca", pca), ("regressor", regressor)])

X_digits, y_digits = X, y
# Parameters of pipelines can be set using ‘__’ separated parameter names:
param_grid = {
    "pca__n_components": [3,5,7,10],
    "regressor__n_estimators": [100,500,700],
    "regressor__max_features": ['auto', 'sqrt', 'log2'],
    "regressor__min_samples_split": [2,3,4,5],
    "regressor__bootstrap": [True,False],
}
search = GridSearchCV(pipe, param_grid, scoring='r2', n_jobs=2)
search.fit(X_digits, y_digits)
print("Best parameter (R2 score=%0.3f):" % search.best_score_)
print(search.best_params_)


Best parameter (CV score=0.172):
{'pca__n_components': 10, 'regressor__bootstrap': True, 'regressor__max_features': 'log2', 'regressor__min_samples_split': 4, 'regressor__n_estimators': 500}


Després de crear un pipeline amb StandardScaler, PCA i Random Forest Regressor, hem fet còrrer el gridSearch amb diferents paràmetres per al PCA i el Random Forest. El gridsearch ens ha donat com a valors óptims: 10 components per al PCA i, bootrsap True, max_features log2, min_samples_split 4 i n_estimators 500 per al Random Forest. Ara podem incloure aquests valors directament al pipeline i fer fit i predict amb el subconjunt test.

In [50]:
pca = PCA(n_components=10)
# Define a Standard Scaler to normalize inputs
scaler = StandardScaler()
# Define regressor
regressor = RandomForestRegressor(bootstrap= True, max_features='log2', min_samples_split= 4, n_estimators= 500)

#Create pipeline
pipe = Pipeline(steps=[("scaler", scaler), ("pca", pca), ("regressor", regressor)])

pipe.fit(X_train,y_train)
y_pred = pd.DataFrame(pipe.predict(X_test), index = X_test.index, columns = y_test.columns)


In [52]:
from sklearn.metrics import r2_score

scores=r2_score(y_test, y_pred)
scores

0.2979473703828758

Tot i que en el subconjunt d'entrenament el R2 score era de 0.172, en el subconjunt test l'score ha millorat lleugerament fins el 0.30. En la taula següent podem veure 10 resultats, la seva predicció i la desviació.

In [59]:
taula = pd.concat([y_test,y_pred], axis=1)[:10]
taula.columns=['y_real','y_pred']
taula['desv.'] = taula['y_pred'] - taula ['y_real']
taula

,y_real,y_pred,desv.
243,5,6.411747,1.411747
478,7,3.498655,-3.501345
433,5,4.373976,-0.626024
426,4,7.637766,3.637766
347,2,4.855310,2.855310
118,5,6.054271,1.054271
489,3,2.791667,-0.208333
98,12,4.465047,-7.534953
290,3,3.211193,0.211193
66,5,3.284043,-1.715957


## Exercici 2. Agafa un text en anglès que vulguis, i calcula'n la freqüència de les paraules

Carreguem alguns documents de fetch_20newsgroups de sklearn.datasets i calculem la freqüència de les paraules d'un dels documents.

In [60]:
from sklearn.datasets import fetch_20newsgroups

In [71]:
docs = fetch_20newsgroups(subset="train").data[1:10]

In [147]:
#Fem una ullada al quart document
docs[4:5]

['From: dfo@vttoulu.tko.vtt.fi (Foxvog Douglas)\nSubject: Re: Rewording the Second Amendment (ideas)\nOrganization: VTT\nLines: 58\n\nIn article <1r1eu1$4t@transfer.stratus.com> cdt@sw.stratus.com (C. D. Tavares) writes:\n>In article <1993Apr20.083057.16899@ousrvr.oulu.fi>, dfo@vttoulu.tko.vtt.fi (Foxvog Douglas) writes:\n>> In article <1qv87v$4j3@transfer.stratus.com> cdt@sw.stratus.com (C. D. Tavares) writes:\n>> >In article <C5n3GI.F8F@ulowell.ulowell.edu>, jrutledg@cs.ulowell.edu (John Lawrence Rutledge) writes:\n>\n>> >> The massive destructive power of many modern weapons, makes the\n>> >> cost of an accidental or crimial usage of these weapons to great.\n>> >> The weapons of mass destruction need to be in the control of\n>> >> the government only.  Individual access would result in the\n>> >> needless deaths of millions.  This makes the right of the people\n>> >> to keep and bear many modern weapons non-existant.\n\n>> >Thanks for stating where you\'re coming from.  Needless to 

### Tokenització

In [169]:
from nltk import RegexpTokenizer

tokenizer = nltk.RegexpTokenizer(r"\w+")
words_dict = tokenizer.tokenize(str(docs[4:5]))

In [170]:
# 20 primers elements del diccionari
words_dict[:20]

['From',
 'dfo',
 'vttoulu',
 'tko',
 'vtt',
 'fi',
 'Foxvog',
 'Douglas',
 'nSubject',
 'Re',
 'Rewording',
 'the',
 'Second',
 'Amendment',
 'ideas',
 'nOrganization',
 'VTT',
 'nLines',
 '58',
 'n']

### Càlcul de la freqüència

In [171]:
from nltk.probability import FreqDist

fdist = FreqDist(words_dict)
fdist.most_common(10)

[('n', 49),
 ('of', 16),
 ('weapons', 13),
 ('the', 12),
 ('to', 10),
 ('mass', 7),
 ('and', 7),
 ('stratus', 6),
 ('com', 6),
 ('destruction', 6)]

Tot i que en el resultat anterior algunes de les paraules més freqüents tenen significat, encara podem fer una neteja per eliminar les que en careixen i, així, obtindre uns resultats que ens permetin esbrinar millor la temàtica del document.

## Nivell 2

## Exercici 1. Treu les stopwords i realitza stemming al teu conjunt de dades.

### Eliminació de stopwords

In [172]:
from nltk.corpus import stopwords
stop_words=set(stopwords.words("english"))

filtered_words_dict=[]
for w in words_dict:
    if w not in stop_words:
            filtered_words_dict.append(w)

### Stemming

In [173]:
from nltk.stem.snowball import SnowballStemmer

#Converting vectorizer vocabulary into stemmer vocabulary
Voc_stemmer = []

stemmer = SnowballStemmer("english") 

for elem in filtered_words_dict:
    raiz = stemmer.stem(str(elem))
    Voc_stemmer.append(raiz)

In [174]:
fdist_2 = FreqDist(Voc_stemmer)
fdist_2.most_common(10)

[('n', 49),
 ('weapon', 13),
 ('destruct', 7),
 ('mass', 7),
 ('stratus', 6),
 ('com', 6),
 ('foxvog', 5),
 ('articl', 5),
 ('write', 5),
 ('vtt', 4)]

Una vegada eliminades les Stopwords i fet el stemming, les paraules més freqüents tenen una mica més de sentit i podem fins i tot deduir que es tracta d'un article de temàtica bèlica, ja que hi trobem les arrels de paraules com arma ('weapon'), destrucció ('destruc'), massa ('mass').

## Nivell 3

## Exercici 1. Realitza sentiment analysis al teu conjunt de dades.


Si fem un analisis ràpid amb el SentimentIntensityAnalyzer de NLTK, ens dona com a resultat un 79% de sentiment neutre, un 18% de negatiu i un 3% de positiu.
Una alternativa es entrenar un model amb altres textos que ja tinguen l'etiqueta de sentiment positiu, negatiu o neutre i llavors introduir aquest text al model per a que faci la predicció.

In [182]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment import SentimentIntensityAnalyzer

sia = SentimentIntensityAnalyzer()
sia.polarity_scores(str(docs[4:5]))

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/yvonnemullermartorell/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


{'neg': 0.18, 'neu': 0.79, 'pos': 0.03, 'compound': -0.9957}